In [1]:
import random
import copy
import numpy as np

In [2]:
def procesar_entrada(direccion_arcivo):
  with open (direccion_arcivo) as entrada:
    datos = entrada.read()
  datos=datos.split('\n')
  n=int(datos[0])
  i=int(datos[1])
  matriz=[]
  datos= datos[2::]
  for j in range(n-1):
    matriz.append([int(datos[j].split(' ')[i]) for i in range(len(datos[j].split(' ')))])
  return(n,i,matriz)

In [3]:
#n,i,matriz = procesar_entrada('entrada.txt')

In [4]:
def obtener_distancia(ciudad1,ciudad2):
  if ciudad2 > ciudad1:
    return(matriz[ciudad1][(ciudad2-ciudad1-1)])
  else:
    return(matriz[ciudad2][ciudad1-ciudad2-1])

In [5]:
def inicial(matriz,n):
  solucion=[0,] #como siempre se inicia en n0, puedo asumir que ya se tiene este elemento al inicio
  solucion.append(matriz[0].index(min(matriz[0]))+1)
  
  for i in range (n-2): #para agregar las ciudades que nos faltan por visitar
    por_visitar= [i for i in range(n) if i not in solucion] #lista con las ciudades que no hemos visitado
    #print(por_visitar)
    menor = 10e100 #aqui guardaremos la menor distancia de la ciudad actual a cualquir otra
    siguiente = None #siguiente ciudad a visitar
    for ciudad in por_visitar:
      distancia= obtener_distancia(solucion[-1],ciudad)
      if distancia < menor:
        menor = distancia
        siguiente = ciudad
    solucion.append(siguiente)
  return(solucion)


In [6]:
#inicial=inicial()

In [7]:
def costo_viaje(solucion):
  global matriz
  costo=0
  for i in range(len(solucion)):
    if i < len(solucion)-1:
      costo +=obtener_distancia(solucion[i],solucion[i+1])
    else:
      costo += obtener_distancia(solucion[i],solucion[0])
  return(costo)

In [8]:
#costo_viaje(inicial)

In [9]:
def vecindario(solucion):
  global n,tabu
  cambiar = random.randint(1,n-1) #selecciona al azar que ciudad cambiar
  x=True
  while x==True: #ciclo para verificar si la ciudad seleccionada no esta en la lista tabu
    no_permitido = [tabu[i][0] for i in range(len(tabu))] #sacamos de la lista tabu las posiciones no permitidas
    if cambiar in no_permitido: #si no esta permitido mover la ciudad, selecciona otra
      cambiar = random.randint(1,n-1)
    else: x = False #si no esta prohibida, rompe el while

  indices_a_cambiar= [i for i in range (n) if i != 0 and i != cambiar] #lista con las n-2 posiciones a donde se va a cambiar la ciudad
  vecindario = []
  for indice in indices_a_cambiar:
    vecino = copy.deepcopy(solucion)
    vecino[cambiar] = solucion[indice]
    vecino[indice] = solucion[cambiar]
    vecindario.append(vecino)
  tabu.append([cambiar,n//2])
  return(vecindario)

In [10]:
#vecindario(inicial)

In [11]:
def actualizar_tabu(tabu):
  tabu = [[tabu[i][0],tabu[i][1]-1] for i in range(len(tabu))]
  for elemento in tabu:
    if elemento[1] == 0:
      tabu.pop(tabu.index(elemento))
  return tabu

In [12]:
def obtener_mediana(m):
  if m %2 == 0:
    costos_ordenados= resultados.sort()
    print(costos_ordenados)
    elementos_medios = resultados.index(costos_ordenados[(m/2)]),resultados.index(costos_ordenados[(m/2-1)])
    return(elementos_medios)


In [21]:
def imprimirResultados(resultados,m):
  ordenados = sorted(resultados, key = lambda x : x[1])
  media = np.mean([ordenados[i][1] for i in range(m)])
  medianas = []
  desviacion = np.std([ordenados[i][1] for i in range(m)])
  print('Mejor viaje encontrado',ordenados[0][0], 'Con un costo de',ordenados[0][1])
  print('Peor viaje enconratrado',ordenados[-1][0], 'Con un costo de',ordenados[-1][1])
  if m %2 == 0:
    medianas.append(ordenados[int(m/2)])
    medianas.append(ordenados[int((m/2)-1)])
    if medianas[0] == medianas[1]: #verificamos que los elementos de la mediana no sean el mismo
      medianas.pop()
      print('Mediana',medianas[0],'Que tiene costo de',medianas[1])
    else:
     print('Medianas',medianas[0][0],'Con costo de',medianas[0][1], 'y tambien',medianas[1][0],'con costo de',medianas[1][1])
  else:
    medianas.append(ordenados[int(m/2)])
    print('Mediana',medianas[0],'Que tiene costo de',medianas[1])
  print('Media del costo de los viajes encontrados',media)
  print('Desviacion estandar de los resultados', desviacion)

In [14]:
def encontrar_mejor(vecinos):
  mejor = vecinos[0]
  costo = costo_viaje(mejor)
  for vecino in vecinos:
    if costo_viaje(vecino) < costo:
      costo = costo_viaje(vecino)
      mejor = vecino
  return([mejor,costo])

In [15]:
def busqueda_tabu(entrada,m):
  global n,i,matriz,tabu,resultados
  n,i,matriz = procesar_entrada(entrada)
  solucion_inicial = inicial(matriz,n)
  resultados = []
  for j in range(m):
    k=0
    tabu=[]
    menor_viaje = solucion_inicial
    menor_costo = costo_viaje(solucion_inicial)
    mejor_vecino = vecindario(solucion_inicial)
    while k < i:
      
      vecinos = vecindario(mejor_vecino[0])
      mejor_vecino = encontrar_mejor(vecinos)
      
      if mejor_vecino[1] < menor_costo:
        menor_costo = mejor_vecino[1]
        menor_viaje  = mejor_vecino[0]
      k += 1
      vecindario1=None
      tabu = actualizar_tabu(tabu)
    resultados.append([menor_viaje,menor_costo])
  #print(resultados)
  imprimirResultados(resultados,m)

In [22]:
busqueda_tabu('entrada.txt',8)

Mejor viaje encontrado [0, 3, 8, 9, 6, 2, 7, 4, 1, 5] Con un costo de 254
Peor viaje enconratrado [0, 5, 3, 8, 9, 6, 2, 7, 1, 4] Con un costo de 271
Medianas [0, 5, 3, 8, 9, 6, 2, 7, 1, 4] Con costo de 271 y tambien [0, 2, 7, 6, 9, 8, 3, 4, 1, 5] con costo de 269
Media del costo de los viajes encontrados 264.375
Desviacion estandar de los resultados 8.061288668693114
